In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import pi, sin, cos, dot

In [30]:
class ThinFilm_model:
    def __init__(self, ambient, thinfilm, substrate):
        self.ambient = ambient
        self.thinfilm = thinfilm
        self.substrate = substrate
    
    def matrix(layer, wavelength):
        delta = 2*pi*(layer['n'])*(layre['d'])/wavelength
        ita = layer['n']
        element = pd.DataFrame(
            {'e1':cos(delta), 'e2':1j/ita*sin(delta), 'e3':1j*ita*sin(delta), 'e4':cos(delta)}
        )
        matrix = np.reshape(element.values.reshape(1,-1), (np.size(wavelength), 2, 2))
        return matrix
    
    def Matrix_dot(layer_up, layer_bot): 
        w, _, _ = np.shape(layer_up)
        eq = [dot(layer_up[i], layer_bot[i]) for i in range(w)]
        return eq

    def Eq_TF_Matrix(self, wavelength):
        eq_matrix = np.identity(2)
        for i in range(np.size(self.thinfilm)):
            eq_matrix = Matrix_dot(matrix(self.thinfilm[-i-1], wavelength), eq_matrix)
        return eq_matrix

    def Eq_admittance(self, wavelength):
        eq = self.Eq_TF_Matrix(wavelength)
        Eq_y = pd.DataFrame(np.reshape(dot(eq, np.array([[1], [self.substrate]])), (np.size(wavelength), 2)), columns = ['B','C'])
        Y = Eq_y['C']/Eq_y['B']
        return Y.values.reshape(np.size(wavelength), 1)
        
    def R(self, wavelength):
        eq_Y = self.Eq_admittance(wavelength)
        r = (self.ambient-eq_Y)/(self.ambient+eq_Y)
        reflectance = np.reshape(r*r.conjugate(), np.size(eq_Y))
        return np.real(reflectance)

In [29]:
model_1 = ThinFilm_model(1, [
    {'n': 1.4, 'd': 100}, 
    {'n': 2.1, 'd': 200}
], 1.5)

model_1.R(np.linspace(900,2000, 1000))

NameError: name 'Matrix_dot' is not defined

In [27]:
for i in model_1.thinfilm:
    print(i)
#print(np.size(model_1.thinfilm))

{'n': 1.4, 'd': 100}
{'n': 2.1, 'd': 200}
